<a href="https://colab.research.google.com/github/parshadk/DataMiningAndBusinessIntelligence/blob/main/apriori%26fpgrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import dataset and

In [1]:
import pandas as pd

df = pd.read_csv('fraud_dataset.csv')
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'fraud_dataset.csv'

In [ ]:
# making a copy
df_selected = df[['type', 'amount', 'isFraud']].copy()

#  'amount' into categorical bins to reduce complexity
df_selected['amount_bin'] = pd.qcut(df_selected['amount'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# 'isFraud' column to string
df_selected['isFraud'] = df_selected['isFraud'].astype(str)

print(df_selected.head())


In [ ]:
# columns we need
columns_to_keep = ['type', 'amount_bin', 'isFraud']

# Convert DataFrame to a list of transactions
transactions = df_selected[columns_to_keep].astype(str).values.tolist()

print("🔹 Sample Transactions:", transactions[:5])


In [ ]:
chunk_size = 10000

transactions = []
for chunk in pd.read_csv('fraud_dataset.csv', chunksize=chunk_size):
    chunk['amount_bin'] = pd.qcut(chunk['amount'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])
    chunk['isFraud'] = chunk['isFraud'].astype(str)
    transactions.extend(chunk[['type', 'amount_bin', 'isFraud']].astype(str).values.tolist())

print("🔹 Processed Transactions:", len(transactions))


In [ ]:
from mlxtend.preprocessing import TransactionEncoder

# Convert transactions to a sparse matrix
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions, sparse=True)

# Convert sparse matrix to DataFrame (saves RAM)
df_trans = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

print("🔹 Encoded Data Shape:", df_trans.shape) # Check memory-efficient encoding



In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# Run Apriori
frequent_itemsets_apriori = apriori(df_trans, min_support=0.01, use_colnames=True)

# Generate rules
rules_apriori = association_rules(frequent_itemsets_apriori, metric="lift", min_threshold=1)

print("🔹 Apriori Frequent Itemsets:\n", frequent_itemsets_apriori.head())
print("\n🔹 Apriori Association Rules:\n", rules_apriori.head())


In [ ]:
from mlxtend.frequent_patterns import fpgrowth

# Run FP-Growth
frequent_itemsets_fpgrowth = fpgrowth(df_trans, min_support=0.05, use_colnames=True)

# Generate rules
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="lift", min_threshold=1)

print("🔹 FP-Growth Frequent Itemsets:\n", frequent_itemsets_fpgrowth.head())
print("\n🔹 FP-Growth Association Rules:\n", rules_fpgrowth.head())
